In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import sys
sys.path.append('../../')

from spicy_snow.retrieval import retrieval_from_parameters

In [ ]:
lidar = []

for fp in Path('../../Lidar_s1_stacks').glob('*.nc'):
    ds = xr.open_dataset(fp)

In [ ]:
ds1 = retrieval_from_parameters(ds, A = 1, B= 1, C = 1)

In [ ]:
ds1['snow_depth'].isel(time = 34).plot()

In [ ]:
ds['lidar-sd']

In [ ]:
ds[['s1','fcf','ims']]